<a href="https://colab.research.google.com/github/piusAI/DeepLearning_Basic/blob/main/Chapter04_Last.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Chapter 04

#### To do this!

- 01 Sigmoid, Softmax, CEE
- 02 Gradient, Sigmoid_gradient(sigmoid differentiation)
- 03 TwoNetwork


Avartar **S**tochastic **D**escent **G**radient


#### 01 sigmoid, Softmax, CEE, Gradient
02 gradient

In [ ]:
import numpy as np

def sigmoid(x):
  return 1 / 1+ np.exp(-x)

def sigmoid_gradient(x):
  return sigmoid(x) * (1 - sigmoid(x) )

def softmax(x):
  if x.ndim==2:
    x= x-np.max(x,axis=1, keepdims=True)
    return np.exp(x) / np.sum(np.exp(x),axis=1, keepdims=True)
  x = x- np.max(x)
  return np.exp(x) / np.sum(np.exp(x))

def CEE(y,t):
  if y.ndim==1:
    y = y.reshape(y.shape[0], 1)
    t = t.reshape(t.shape[0], 1)
  batch_size = y
  return -np.sum(y[np.arange(batch_size), t])/ batch_size

def _numerical_gradient_no_batch(f,x):
  h=1e-4
  grad = np.zeros_like(x)

  for idx in range(x.size):
    tmp_val = x[idx]
    x[idx] = float(tmp_val)+h
    fxh1 = f(x)

    x[idx] = tmp_val - h
    fxh2 = f(x)

    grad[idx] = (fxh1-fxh2) / ( 2* h)
    x[idx] = tmp_val
    return grad
def numerical_gradient(f,X):
  if X.ndim == 1:
    return _numerical_gradient_no_batch(f,X)
  else:
    grad = np.zeros_like(X)
    for i, idx in enumerate(X):
      grad[i] = _numerical_gradient_no_batch(f,idx)
    return grad

####03 Two Network

In [ ]:
class TwolayerNet:
  def __init__(self, input_size, hidden_size, output_size, weight_init_std= 0.01):
    self.params = {}
    self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
    self.params['b1'] = np.zeros(hidden_size)
    self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
    self.params['b2'] = np.zeros(output_size)
  def predict(self, x):
    W1, W2 = self.params['W1'], self.params['W2']
    b1, b2 = self.params['b1'], self.params['b2']

    a1 = np.dot(x, W1) + b1
    y1 = sigmoid(a1)
    a2 = np.dot(y1, W2) + b2
    y2 = softmax(a2)
    return y2
  def accuracy(self, x, t):
    y = self.predict(x)
    y = np.argmax(y, axis=1)

    if t.ndim != 1:
      t = np.argmax(t, axis=1)

    accuracy = np.sum(y == t) / float(x.shape[0])
    return accuracy

  def loss(self, x, t):
    y = self.predict(x)
    return CEE(y,t)

  def numerical_gradient(self, x, t):
    loss_w = lambda w : self.loss(x,t)

    grads = {}
    grads['W1'] = numerical_gradient(loss_w, self.params['W1'])
    grads['b1'] = numerical_gradient(loss_w, self.params['b1'])
    grads['W2'] = numerical_gradient(loss_w, self.params['W2'])
    grads['b2'] = numerical_gradient(loss_w, self.params['b2'])

    return grads

#역전파는 그냥 Pass
  def gradient(self, x, t):
    W1, W2 = self.params['W1'], self.params['W2']
    b1, b2 = self.params['b1'], self.params['b2']

    grads={}

    batch_num = x.shape[0]
    a1= np.dot(x,W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    y = softmax(a2)

    if t.ndim ==1:
      t_one_hot = np.zeros_like(y)
      t_one_hot[np.arange(batch_size), t] = 1
      t = t_one_hot
    dy = (y-t) /batch_num
    grads ['W2'] = np.dot(z1.T, dy)
    grads['b2'] = np.sum(dy, axis=0)

    dz1 = np.dot(dy, W2.T)
    da1 = sigmoid_gradient(a1) * dz1
    grads['W1'] = np.dot(x.T, da1)
    grads['b1'] = np.sum(da1, axis=0)

    return grads